# Практическое задание №2 по теме "Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)".

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pymorphy2
import itertools
import nltk

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.test.utils import datapath
from nltk.corpus import stopwords
from razdel import tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

%matplotlib inline

In [2]:
result_dict = {'Mean':[], 'Median':[], 'Max':[]} 

## Задание №1

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html <br>
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

С информацией по ссылкам ознакомился.

## Задание №2

#### Формирование векторного представления новостей и инициализация данных

1. Загрузка данных:

In [3]:
news = pd.read_csv("materials.csv")
users = pd.read_csv("users_articles.csv")
target = pd.read_csv("users_churn.csv")

In [4]:
# nltk.download('stopwords')
stopword_ru = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

In [5]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords

2. Предобработка:

In [6]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация (морфоанализатор)
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова 
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [7]:
%%time
# Очистка текста
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

C:\Temp\ipykernel_2020\974220112.py:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: total: 20.6 s
Wall time: 20.6 s


In [8]:
%%time
# Лемматизация очищенного текста
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: total: 2min 35s
Wall time: 2min 35s


3. Словарь слов из обработанных данных:

In [9]:
texts = [t for t in news['title'].values]
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

4. LDA:

In [10]:
N_topics = 25 # 25
N_words = 7 

In [11]:
%%time
lda = LdaModel(common_corpus, num_topics=N_topics, id2word=common_dictionary)

CPU times: total: 26.2 s
Wall time: 26.3 s


In [12]:
# Сохранение LDA модели.
temp_file = datapath("model.lda")
lda.save(temp_file)
lda = LdaModel.load(temp_file)

In [13]:
x = lda.show_topics(num_topics=N_topics, num_words=N_words, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

for topic, words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: человек это который год ребёнок обнаружить тело
topic_1: статья nn год писать москва исследование который
topic_2: год млрд исследование рост млн рынок китай
topic_3: квартира температура выяснить журнал градус отдых констатировать
topic_4: сон лесной сергеев норматив калининградский браун азербайджан
topic_5: газ турция турецкий офицер фильм превысить высота
topic_6: год который nn смерть время стать это
topic_7: год компания экономика журнал млн который цена
topic_8: продукция энергия сотрудничать сближение пищевой хабаровский индустрия
topic_9: это который мочь свой всё говорить год
topic_10: медведев определение музыка лодка эксперимент студия орден
topic_11: мозг исследование поверхность пациент день врач лечение
topic_12: который операция nn это научный космос человек
topic_13: который год это страна россия nn новый
topic_14: рубль банк год который дело суд это
topic_15: год россия украина это сша гражданин который
topic_16: египет ступень бензин билет фрагмент школьный 

5. Векторное представление новостей:

In [14]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(N_topics):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [15]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(N_topics)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(N_topics)]]

In [16]:
# Словарь новостей в векторном представлении
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(N_topics)]].values))

#### Обучение модели и вычисление метрик для get_user_embedding на основе среднего (mean)

In [17]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

Векторное представление пользователей

In [18]:
user_embeddings_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
user_embeddings_mean.columns = ['topic_{}'.format(i) for i in range(N_topics)]
user_embeddings_mean['uid'] = users['uid'].values
user_embeddings_mean = user_embeddings_mean[['uid']+['topic_{}'.format(i) for i in range(N_topics)]]

Тренировочный датасет

In [19]:
X_mean = pd.merge(user_embeddings_mean, target, 'left')

Обучение и предсказание

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_mean[['topic_{}'.format(i) for i in range(N_topics)]], 
                                                    X_mean['churn'], random_state=0)

In [21]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [22]:
preds_mean = logreg.predict_proba(X_test)[:, 1]

Вычисление метрик и оптимального порога

In [23]:
ras = roc_auc_score(y_test, preds_mean)
precision, recall, thresholds = precision_recall_curve(y_test, preds_mean)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

Сохранение полученных результатов для "mean"

In [24]:
result_dict['Mean'].append(thresholds[ix])
result_dict['Mean'].append(fscore[ix])
result_dict['Mean'].append(precision[ix])
result_dict['Mean'].append(recall[ix])
result_dict['Mean'].append(ras)

#### Обучение модели и вычисление метрик для get_user_embedding на основе медианы (median)

In [25]:
def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

Векторное представление пользователей

In [26]:
user_embeddings_median = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings_median.columns = ['topic_{}'.format(i) for i in range(N_topics)]
user_embeddings_median['uid'] = users['uid'].values
user_embeddings_median = user_embeddings_median[['uid']+['topic_{}'.format(i) for i in range(N_topics)]]

Тренировочный датасет

In [27]:
X_median = pd.merge(user_embeddings_median, target, 'left')

Обучение и предсказание

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_median[['topic_{}'.format(i) for i in range(N_topics)]], 
                                                    X_median['churn'], random_state=0)

In [29]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [30]:
preds_median = logreg.predict_proba(X_test)[:, 1]

Вычисление метрик и оптимального порога

In [31]:
ras = roc_auc_score(y_test, preds_median)
precision, recall, thresholds = precision_recall_curve(y_test, preds_median)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

Сохранение полученных результатов для "median"

In [32]:
result_dict['Median'].append(thresholds[ix])
result_dict['Median'].append(fscore[ix])
result_dict['Median'].append(precision[ix])
result_dict['Median'].append(recall[ix])
result_dict['Median'].append(ras)

## Задание №3

#### Обучение модели и вычисление метрик для get_user_embedding на основе максимума (max)

In [33]:
def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

Векторное представление пользователей

In [34]:
user_embeddings_max = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings_max.columns = ['topic_{}'.format(i) for i in range(N_topics)]
user_embeddings_max['uid'] = users['uid'].values
user_embeddings_max = user_embeddings_max[['uid']+['topic_{}'.format(i) for i in range(N_topics)]]

Тренировочный датасет

In [35]:
X_max = pd.merge(user_embeddings_max, target, 'left')

Обучение и предсказание

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_max[['topic_{}'.format(i) for i in range(N_topics)]], 
                                                    X_max['churn'], random_state=0)

In [37]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [38]:
preds_max = logreg.predict_proba(X_test)[:, 1]

Вычисление метрик и оптимального порога

In [39]:
ras = roc_auc_score(y_test, preds_max)
precision, recall, thresholds = precision_recall_curve(y_test, preds_max)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

Сохранение полученных результатов для "max"

In [40]:
result_dict['Max'].append(thresholds[ix])
result_dict['Max'].append(fscore[ix])
result_dict['Max'].append(precision[ix])
result_dict['Max'].append(recall[ix])
result_dict['Max'].append(ras)

## Задание №4

Дополнительное задание, решил его не выпонлять.

## Задание №5

N_topics = 15 

In [41]:
result_df_15 = pd.DataFrame(result_dict, index=['Best Threshold','F-Score','Precision','Recall','Roc_Auc_Score'])
result_df_15

,Mean,Median,Max
Best Threshold,0.257136,0.290724,0.323220
F-Score,0.685185,0.754941,0.818004
Precision,0.627119,0.731801,0.785714
Recall,0.755102,0.779592,0.853061
Roc_Auc_Score,0.952309,0.967200,0.976959


N_topics = 25 

In [42]:
result_df_25 = pd.DataFrame(result_dict, index=['Best Threshold','F-Score','Precision','Recall','Roc_Auc_Score'])
result_df_25

,Mean,Median,Max
Best Threshold,0.259007,0.284328,0.335070
F-Score,0.666667,0.715356,0.765873
Precision,0.586957,0.660900,0.745174
Recall,0.771429,0.779592,0.787755
Roc_Auc_Score,0.946364,0.956688,0.968335


## Задание №6

Прогнал ноутбук для двух разных значений гиперпараметра "количество тем": 15 и 25. 

Получилось что по большинству показателей для обоих вариантов лидером оказался метод **Max**.

С чем это может быть связано? По сути по методу **Max** в каждом топике вероятность будет соответствовать максимальной вероятности, какая только может быть из статей, которые читал пользователь. Соответственно, разница в вероятностях между темами, которые были ярко выражены в прочитанной пользователем статье, и темами, которые в прочитанных статьях почти не фигурировали, будет довольно большой (серьезный разрыв в показателях вероятности между topics) по сравнению с разрывом в показателях вероятности между топиками расчитаным по методу **Mean**, **Median**. Из-за такого контраста, получается, что все прочитанные топики пользователем выражены ярче, отсюда скорее всего и более лучшие показатели метрик по методу **Max**.

Что касается порога, то очевидно по методу **Max** он будет больше, так как полученные значения вероятности по каждому топику в среднем будут выше.  

---